In [1]:
import requests
import os
import pandas as pd
from IPython.core.display import HTML
import json
from cred import credentials

In [2]:
# to connect to the Genesis API via data/table endpoint
de_request_url = f'https://www-genesis.destatis.de/genesisWS/rest/2020/data/table'
de_request_params = {
    'username': "username",
    'password': "password",
    "name": "12421-0004",
    "area": "all",
    "compress": "false",
    "transpose": "false",
    "startyear": "",
    "endyear": "",
    "timeslices": "",
    "regionalvariable": "",
    "regionalkey": "",
    "classifyingvariable1": "",
    "classifyingkey1": "",
    "classifyingvariable2": "",
    "classifyingkey2": "",
    "classifyingvariable3": "",
    "classifyingkey3": "",
    "job": "false",
    "stand": "01.01.1970",
    "language": "en" ,
}
# save the response
de_response = requests.get(url=de_request_url, params=de_request_params)
# to save the response as json in a file called "r_population.json" 
with open('data/r_population.json', 'w') as f:
    # json.dump(de_response.json(), f, indent=2)
    f.write(json.dumps(de_response.json(), indent=2))
# to save the part of response (the table) as a csv called "r_population.csv" 
with open('data/r_population.csv', 'w') as csv_file:
    csv_file.write(de_response.json()['Object']['Content'])

KeyError: 'Object'

In [2]:
# define a function to request the Genesis API 
def get_genesis_api(
        code: str,
        years: list[int],
        skiprows: int=4,
        skipfooter: int=4,
        delimiter: str=';',
        header: list[int]=[0,1],
        save_json: bool=False,
) -> pd.DataFrame:

    dataframes = {}
    for year in years:
        de_request_url = f'https://www-genesis.destatis.de/genesisWS/rest/2020/data/table'
        de_request_params = {
            'username': credentials['username'],
            'password': credentials['password'],
            "name": code,  # e.g. "12411-0014" the population per state, sex, nationality
            "area": "all",
            "compress": "false",
            "transpose": "false",
            "startyear": year,
            "endyear": year,
            "timeslices": "",
            "regionalvariable": "",
            "regionalkey": "",
            "classifyingvariable1": "",
            "classifyingkey1": "",
            "classifyingvariable2": "",
            "classifyingkey2": "",
            "classifyingvariable3": "",
            "classifyingkey3": "",
            "job": "false",
            "stand": "01.01.1970",
            "language": "en" ,
        }
        # to save the response as json in a file called "r_population_kries_year.json"
        de_response = requests.get(url=de_request_url, params=de_request_params)
        if save_json:
            with open(f'data/{code}_{year}.json', 'w') as f:
                json.dump(de_response.json(), f, indent=2)

        # to save the part of response (the table) as a csv called "r_population_kreis_year.csv" 
        with open(f'data/{code}_{year}.csv', 'w') as csv_file:
            csv_file.write(de_response.json()['Object']['Content'])

        dataframes[year] = pd.read_csv(f'data/{code}_{year}.csv', skiprows=skiprows, skipfooter=skipfooter, delimiter=delimiter, header=header)  
    
    result = pd.concat(dataframes.values(), axis=0)
    result.to_csv(f'data/{code}_{min(years)}_{max(years)}.csv')
    return result



In [57]:
get_genesis_api('12421-0004', [2030])

NameError: name 'get_genesis_api' is not defined

In [91]:
population_prediction_2030=pd.read_csv("data/12421-0004_2030_2030.csv", header= [0,1])

population_prediction_2030

,Unnamed: 0_level_0,Projected population figures (1000),Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0
,Unnamed: 0_level_1,Unnamed: 0_level_1,Unnamed: 1_level_1,Unnamed: 2_level_1,Unnamed: 3_level_1,Unnamed: 4_level_1,Reference date
0,0,NaN,NaN,NaN,NaN,NaN,2030-12-31
1,1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,under 1 year,52
2,2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1 year,53
3,3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2 years,54
4,4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3 years,55
...,...,...,...,...,...,...,...
14684,14684,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,97 years,1
14685,14685,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,98 years,1
14686,14686,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,99 years,0


In [92]:
population_prediction_2030.columns=["num", "state", "scenarios", "assumptions", "sex", "age", "estimated_population"]
population_prediction_2030

,num,state,scenarios,assumptions,sex,age,estimated_population
0,0,NaN,NaN,NaN,NaN,NaN,2030-12-31
1,1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,under 1 year,52
2,2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1 year,53
3,3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2 years,54
4,4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3 years,55
...,...,...,...,...,...,...,...
14684,14684,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,97 years,1
14685,14685,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,98 years,1
14686,14686,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,99 years,0
14687,14687,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,100 years and over,1


In [93]:
population_prediction_2030 = population_prediction_2030.assign(age = lambda x: x['age'].str.extract('(\d+)'))
population_prediction_2030

,num,state,scenarios,assumptions,sex,age,estimated_population
0,0,NaN,NaN,NaN,NaN,NaN,2030-12-31
1,1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,52
2,2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,53
3,3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2,54
4,4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3,55
...,...,...,...,...,...,...,...
14684,14684,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,97,1
14685,14685,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,98,1
14686,14686,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,99,0
14687,14687,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,100,1


In [94]:
population_prediction_2030["country"]= "Germany"
population_prediction_2030["Date"]= "2030"
population_prediction_2030

,num,state,scenarios,assumptions,sex,age,estimated_population,country,Date
0,0,NaN,NaN,NaN,NaN,NaN,2030-12-31,Germany,2030
1,1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,52,Germany,2030
2,2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,53,Germany,2030
3,3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2,54,Germany,2030
4,4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3,55,Germany,2030
...,...,...,...,...,...,...,...,...,...
14684,14684,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,97,1,Germany,2030
14685,14685,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,98,1,Germany,2030
14686,14686,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,99,0,Germany,2030
14687,14687,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,100,1,Germany,2030


In [95]:
population_prediction_2030 = population_prediction_2030.drop('num', 1)


/var/folders/2j/1gvl6by90hj5nl8__0216fxm0000gn/T/ipykernel_1932/420686967.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  population_prediction_2030 = population_prediction_2030.drop('num', 1)


In [96]:
population_prediction_2030 = population_prediction_2030[population_prediction_2030.estimated_population != "2030-12-31"]
population_prediction_2030

,state,scenarios,assumptions,sex,age,estimated_population,country,Date
1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,52,Germany,2030
2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,53,Germany,2030
3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2,54,Germany,2030
4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3,55,Germany,2030
5,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,4,56,Germany,2030
...,...,...,...,...,...,...,...,...
14684,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,97,1,Germany,2030
14685,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,98,1,Germany,2030
14686,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,99,0,Germany,2030
14687,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Total,100,1,Germany,2030


In [97]:
population_prediction_2030 = population_prediction_2030[population_prediction_2030.sex != "Total"]
population_prediction_2030

,state,scenarios,assumptions,sex,age,estimated_population,country,Date
1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,52,Germany,2030
2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,53,Germany,2030
3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2,54,Germany,2030
4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3,55,Germany,2030
5,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,4,56,Germany,2030
...,...,...,...,...,...,...,...,...
14582,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,97,1,Germany,2030
14583,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,98,1,Germany,2030
14584,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,99,0,Germany,2030
14585,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,100,1,Germany,2030


In [98]:
population_prediction_2030.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9792 entries, 1 to 14586
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state                 9792 non-null   object
 1   scenarios             9792 non-null   object
 2   assumptions           9792 non-null   object
 3   sex                   9792 non-null   object
 4   age                   9696 non-null   object
 5   estimated_population  9792 non-null   object
 6   country               9792 non-null   object
 7   Date                  9792 non-null   object
dtypes: object(8)
memory usage: 688.5+ KB


In [99]:
population_prediction_2030['estimated_population'] = pd.to_numeric(population_prediction_2030['estimated_population'])
#population_prediction_2030['age'] = pd.to_numeric(population_prediction_2030['age'])

#population_prediction_2030['age'].astype(int)

/var/folders/2j/1gvl6by90hj5nl8__0216fxm0000gn/T/ipykernel_1932/1162977625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_prediction_2030['estimated_population'] = pd.to_numeric(population_prediction_2030['estimated_population'])


In [100]:
population_prediction_2030.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9792 entries, 1 to 14586
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state                 9792 non-null   object
 1   scenarios             9792 non-null   object
 2   assumptions           9792 non-null   object
 3   sex                   9792 non-null   object
 4   age                   9696 non-null   object
 5   estimated_population  9792 non-null   int64 
 6   country               9792 non-null   object
 7   Date                  9792 non-null   object
dtypes: int64(1), object(7)
memory usage: 688.5+ KB


In [101]:
population_prediction_2030.isnull().sum(axis=0).tolist()

[0, 0, 0, 0, 96, 0, 0, 0]

In [102]:
population_prediction_2030[population_prediction_2030["age"].isnull()]

,state,scenarios,assumptions,sex,age,estimated_population,country,Date
102,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,NaN,5670,Germany,2030
204,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Female,NaN,5669,Germany,2030
408,Baden-Württemberg,BEV-VARIANTE-02,"Mod. birth rate, life expectancy, mig.bal.(G2L...",Male,NaN,5692,Germany,2030
510,Baden-Württemberg,BEV-VARIANTE-02,"Mod. birth rate, life expectancy, mig.bal.(G2L...",Female,NaN,5687,Germany,2030
714,Baden-Württemberg,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Male,NaN,5780,Germany,2030
...,...,...,...,...,...,...,...,...
13974,Thüringen,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Female,NaN,1000,Germany,2030
14178,Thüringen,BEV-VARIANTE-02,"Mod. birth rate, life expectancy, mig.bal.(G2L...",Male,NaN,989,Germany,2030
14280,Thüringen,BEV-VARIANTE-02,"Mod. birth rate, life expectancy, mig.bal.(G2L...",Female,NaN,1002,Germany,2030
14484,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Male,NaN,999,Germany,2030


In [103]:
population_prediction_2030= population_prediction_2030.dropna(axis=0)


In [104]:
population_prediction_2030.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9696 entries, 1 to 14585
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state                 9696 non-null   object
 1   scenarios             9696 non-null   object
 2   assumptions           9696 non-null   object
 3   sex                   9696 non-null   object
 4   age                   9696 non-null   object
 5   estimated_population  9696 non-null   int64 
 6   country               9696 non-null   object
 7   Date                  9696 non-null   object
dtypes: int64(1), object(7)
memory usage: 681.8+ KB


In [105]:
population_prediction_2030["population"]= population_prediction_2030["estimated_population"]*1000
population_prediction_2030

/var/folders/2j/1gvl6by90hj5nl8__0216fxm0000gn/T/ipykernel_1932/743546951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_prediction_2030["population"]= population_prediction_2030["estimated_population"]*1000


,state,scenarios,assumptions,sex,age,estimated_population,country,Date,population
1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,52,Germany,2030,52000
2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,53,Germany,2030,53000
3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2,54,Germany,2030,54000
4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3,55,Germany,2030,55000
5,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,4,56,Germany,2030,56000
...,...,...,...,...,...,...,...,...,...
14581,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,96,1,Germany,2030,1000
14582,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,97,1,Germany,2030,1000
14583,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,98,1,Germany,2030,1000
14584,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,99,0,Germany,2030,0


In [106]:
population_prediction_2030 = population_prediction_2030.drop('estimated_population', 1)


/var/folders/2j/1gvl6by90hj5nl8__0216fxm0000gn/T/ipykernel_1932/1123204820.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  population_prediction_2030 = population_prediction_2030.drop('estimated_population', 1)


In [107]:
population_prediction_2030

,state,scenarios,assumptions,sex,age,country,Date,population
1,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,Germany,2030,52000
2,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,1,Germany,2030,53000
3,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,2,Germany,2030,54000
4,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,3,Germany,2030,55000
5,Baden-Württemberg,BEV-VARIANTE-01,"Mod. birth rate a. life exp.,low mig.bal. (G2L...",Male,4,Germany,2030,56000
...,...,...,...,...,...,...,...,...
14581,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,96,Germany,2030,1000
14582,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,97,Germany,2030,1000
14583,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,98,Germany,2030,1000
14584,Thüringen,BEV-VARIANTE-03,"Mod. birth rate a. life exp.,high mig.bal.(G2L...",Female,99,Germany,2030,0


In [108]:
from sql import engine

In [109]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'population_prediction_2030'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        population_prediction_2030.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        # schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The population_prediction_2030 table was imported successfully.
